Import des libs

In [2]:
import numpy as np
import pandas as pd
import os
import shapely
# libs nécessaires pour requêter BigQuery
from google.cloud import bigquery
from google.oauth2 import service_account



In [ ]:
# Mise en place des credentials BigQuery

# définition de l'emplacement de la clé du compte de service Google
credpath = os.path.join("..", "credentials", "artefact-da53-projet-final-b60d2589fda1.json")

# définition des credentials Google en variable d'environnement en pointant vers la clé du compte de service Google
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credpath

# préparation et définition des credentials du compte de service Google en pointant vers la clé JSON et en définissant les scopes d'action : drive (on a des tables à base de Google Sheets), cloud plateform, et bigquery.
credentials = service_account.Credentials.from_service_account_file(credpath,  scopes=["https://www.googleapis.com/auth/drive","https://www.googleapis.com/auth/cloud-platform","https://www.googleapis.com/auth/bigquery"])

In [ ]:
# Import BigQuery

# préparation du client bigquery avec appel à la fonction Client, en faisant passer les infos de credentials définies plus haut.
client = bigquery.Client(credentials=credentials, project=credentials.project_id)


# préparation de la requête SQL qui sera utilisée dans bigquery
query = """SELECT *
        FROM `artefact-da53-projet-final.02_Transport.2019_2023_circulation_paris_bp_pim_raw`"""

# récupération dans une dataframe des résultats de la requête BQ
df = client.query(query).to_dataframe()

df.head(10)

/home/tonychamcham/.pyenv/versions/Artefact/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,annee,VOLUME_PIM,VITESSE_PIM,VOLUME_BP,VITESSE_BP,KPIM_ANNUEL,VPIM_ANNUEL,KBP_ANNUEL,VPBP_ANNUEL
0,2019-09-01,694.0,12.3,5235,33.799225,683.00,12.30,5146.000000,34.500000
1,2019-10-01,686.7,12.6,5169,32.780596,683.00,12.30,5146.000000,34.500000
2,2020-09-01,670.9,12.5,5358,35.400000,571.00,13.40,4828.000000,40.400000
3,2022-06-01,626.3,11.8,4854,31.100000,585.21,12.43,4918.666667,36.361667
4,2023-12-01,518.6,10.4,4768,34.600000,546.00,11.30,4814.000000,34.100000
5,2021-04-01,567.5,14.3,5267,47.900000,600.00,12.60,5063.000000,37.300000
6,2022-10-01,573.5,12.6,4863,37.300000,585.21,12.43,4918.666667,36.361667
7,2023-02-01,566.9,12.0,4949,34.300000,546.00,11.30,4814.000000,34.100000
8,2021-10-01,634.0,11.6,4920,31.600000,600.00,12.60,5063.000000,37.300000
9,2022-03-01,621.8,12.4,5031,35.200000,585.21,12.43,4918.666667,36.361667


In [ ]:
# Export BigQuery - Création d'un nouveau dataset, d'une nouvelle table, et remplissage de la table avec un dataframe Pandas

# dataframe de test
data = {
    'Numberzzzz': [1, 2, 10],
    'Noms': ['A', 'B', 'C']
}
df = pd.DataFrame(data)

# Init the BQ client
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

# Define the dataset id
dataset_id = 'artefact-da53-projet-final.testDataset'

# Create a Dataset object
dataset = bigquery.Dataset(dataset_id)

# Create the dataset
dataset = client.create_dataset(dataset, exists_ok=True)

print(f'Dataset créé : {client.project}.{dataset.dataset_id}')

# Define the table id
table_id = 'artefact-da53-projet-final.testDataset.testTable'

# Create a Table object
table = bigquery.Table(table_id)

# Create a table
table = client.create_table(table, exists_ok=True)

print(f'Table créée : {client.project}.{dataset.dataset_id}.{table.table_id}')


# envoi du dataframe vers BQ
df.to_gbq(destination_table=table_id, if_exists='replace')




Dataset créé : artefact-da53-projet-final.testDataset
Table créée : artefact-da53-projet-final.testDataset.testTable


/tmp/ipykernel_409057/1484279815.py:37: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(destination_table=table_id, if_exists='replace')
100%|██████████| 1/1 [00:00<00:00, 2896.62it/s]
